In [5]:
# CRL_UTF.origの前処理
INPUT_FILE = "corpus/CRL_UTF.orig"
OUTPUT_FILE = "data/P_CRL.txt"
texts = []
with open("corpus/CRL_UTF.orig") as f:
    texts = [line.strip() for line in f.readlines()]
texts[:5]

['# S-ID:950101003-001',
 '村山富市首相は年頭にあたり首相官邸で内閣記者会と二十八日会見し、社会党の新民主連合所属議員の離党問題について「政権に影響を及ぼすことにはならない。離党者がいても、その範囲にとどまると思う」と述べ、大量離党には至らないとの見通しを示した。',
 '# S-ID:950101003-002',
 'また、一九九五年中の衆院解散・総選挙の可能性に否定的な見解を表明、二十日召集予定の通常国会前の内閣改造を明確に否定した。',
 '# S-ID:950101004-002']

In [6]:
import re
import os

In [7]:
os.makedirs("data", exist_ok=True)

In [8]:
# 文章ごとにリストに格納
pattern = re.compile(pattern=r"#\sS-ID:(\d+)-\d+")
prev_num = pattern.findall(texts[0])[0]
sentences = [""]
tmp_text = ""
for i in range(0, len(texts) - 1, 2):
    if not prev_num == pattern.findall(texts[i])[0]:
        sentences.append("")
    prev_num = pattern.findall(texts[i])[0]
    sentences[len(sentences) - 1] += texts[i + 1]


In [9]:
# 文章ごとにMeCabで構文解析を行いファイルに格納
import MeCab

In [10]:
mecab = MeCab.Tagger("-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
mecab.parse("")

'\n'

In [11]:
with open(OUTPUT_FILE, mode="w") as f:
    for sentence in sentences:
        f.write(mecab.parse(sentence) + "\n")

In [12]:
# CRL_NE_DATA_UTF.idxの前処理
INPUT_FILE = "corpus/CRL_NE_DATA_UTF.idx"
OUTPUT_FILE = "data/P_CRL_TAG.txt"
texts = []
with open(INPUT_FILE) as f:
    texts = [line.strip() for line in f.readlines()[14:]]
texts[:10]

['@ 0 4 <PERSON> </PERSON> 村山富市',
 '@ 7 9 <DATE> </DATE> 年頭',
 '@ 13 17 <LOCATION> </LOCATION> 首相官邸',
 '@ 18 23 <ORGANIZATION> </ORGANIZATION> 内閣記者会',
 '@ 24 28 <DATE> </DATE> 二十八日',
 '@ 32 35 <ORGANIZATION> </ORGANIZATION> 社会党',
 '@ 36 41 <ORGANIZATION> </ORGANIZATION> 新民主連合',
 '@ 121 126 <DATE> </DATE> 一九九五年',
 '@ 128 130 <ORGANIZATION> </ORGANIZATION> 衆院',
 '@ 151 154 <DATE> </DATE> 二十日']

In [19]:
pattern = re.compile(pattern=r"@\s(\d+)\s(\d+)\s<.+>\s</(\w+)>\s(.+)")
with open(OUTPUT_FILE, mode="w") as f:
    for line in texts:
        if pattern.match(line):
            start, end, tag, entity = pattern.match(line).groups()
            f.write(f"{start} {int(end) - 1} {tag} {entity}\n")
        else:
            f.write("\n")

In [35]:
# 作成した2つのファイルから正解ラベルのデータを作成する
INPUT_CRL = "data/P_CRL.txt"
INPUT_TAG = "data/P_CRL_TAG.txt"
OUTPUT_FILE = "data/LABEL.txt"

# crl_texts と tags の同じインデックスは同じ文書を表している

crl_texts = []
with open(INPUT_CRL) as f:
    crl_texts = [line.strip() for line in f.readlines() if line != "\n"]

# 二次配列
tags = []
with open(INPUT_TAG) as f:
    tmp_tags = []
    for line in f.readlines():
        if line == "\n":
            tags.append(tmp_tags)
            tmp_tags = []
        else:
            start, end, tag, entity = line.strip().split()
            tmp_tags.append((int(start), int(end), tag))
    tags.append(tmp_tags)

In [48]:
with open(OUTPUT_FILE, mode="w") as f:
    for i in range(len(tags)):
        total_length = 0
        tokens = crl_texts[i].split()
        # タグの始まりのインデックスのリスト
        starts = [elem[0] for elem in tags[i]]
        j = 0
        while j < len(tokens):
            if not total_length in starts:
                f.write(f"{tokens[j]} O\n")
                total_length += len(tokens[j])
                j += 1
            else:
                l_i = starts.index(total_length)
                f.write(f"{tokens[j]} B_{tags[i][l_i][2]}\n")
                total_length += len(tokens[j])
                j += 1
                end = tags[i][l_i][1]
                while total_length <= end:
                    f.write(f"{tokens[j]} I_{tags[i][l_i][2]}\n")
                    total_length += len(tokens[j])
                    j += 1
        f.write("\n")
        